# Preparation

In [ ]:
import numpy as np
import IPython.display
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.animation
from PIL import Image

import scipy
from scipy import ndimage

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
import torch.nn.functional as F

def figure_world(A, cmap='viridis'):
  global img
  fig = plt.figure()
  img = plt.imshow(A, cmap=cmap, interpolation="nearest", vmin=0)
  plt.close()
  return fig

def figure_asset(K, growth, cmap='viridis', K_sum=1, bar_K=False):
  global R
  K_size = K.shape[0];  K_mid = K_size // 2
  fig, ax = plt.subplots(1, 3, figsize=(14,2), gridspec_kw={'width_ratios': [1,1,2]})
  ax[0].imshow(K, cmap=cmap, interpolation="nearest", vmin=0)
  ax[0].title.set_text('kernel K')
  if bar_K:
    ax[1].bar(range(K_size), K[K_mid,:], width=1)
  else:
    ax[1].plot(range(K_size), K[K_mid,:])
  ax[1].title.set_text('K cross-section')
  ax[1].set_xlim([K_mid - R - 3, K_mid + R + 3])
  if K_sum <= 1:
    x = np.linspace(0, K_sum, 1000)
    ax[2].plot(x, growth(x))
  else:
    x = np.arange(K_sum + 1)
    ax[2].step(x, growth(x))
  ax[2].axhline(y=0, color='grey', linestyle='dotted')
  ax[2].title.set_text('target T')
  return fig

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
DTYPE  = torch.float32
torch.set_default_dtype(DTYPE)


def t(x, *, dtype=DTYPE):
    return torch.as_tensor(x, dtype=dtype, device=DEVICE)

In [ ]:
def bell(x, m, s):
    return np.exp(-((x - m) / s) ** 2 / 2)
def target(U):
  return bell(U, m, s)
def update(i):
  global A, img
  U = np.real(np.fft.ifft2(fK * np.fft.fft2(A)))
  A = A + 1/T * (target(U) - A)
  img.set_array(A)
  return img

# Modified for changing m,s and fK.
def update_param(i):
  global A, img
  U = np.real(np.fft.ifft2(fK * np.fft.fft2(A)))
  A = A + 1/T * (bell(U,_m,_s) - A)
  img.set_array(A)
  return img
def update_param_all(i):
  global A, img
  U = np.real(np.fft.ifft2(fK_new * np.fft.fft2(A)))
  A = A + 1/T * (bell(U,_m,_s) - A)
  img.set_array(A)
  return img

# Functions for torch implementation

def bell_torch(x, m, s):
    return torch.exp(-((x - m) / s) ** 2 / 2)

def target_torch(U):
    return bell_torch(U, 0.21, 0.018)


def kernel_fft(b, R, size, _sigma=0.15, *, device=DEVICE, dtype=torch.float32):
    b = b.to(device=device, dtype=dtype)
    g = torch.arange(size, device=device, dtype=dtype) - size // 2
    X, Y = torch.meshgrid(g, g, indexing='ij')
    D = torch.sqrt(X * X + Y * Y) / R * b.numel()

    idx = torch.clamp(D.long(), max=b.numel() - 1)
    K = (D < b.numel()).float() * b[idx] * torch.exp(-((D % 1.0 - 0.5) ** 2) / (2 * _sigma ** 2))
    K /= K.sum()

    K_fft = torch.fft.fft2(torch.fft.ifftshift(K))
    return K_fft


def grad_periodic(__A):
    _A_x = 0.5 * (torch.roll(__A, 1, 1) - torch.roll(__A, -1, 1))
    _A_y = 0.5 * (torch.roll(__A, 1, 0) - torch.roll(__A, -1, 0))
    return _A_x, _A_y

def Loss_torch(__A, _v):
    _gradA_x = grad_periodic(__A)[0]
    _gradA_y = grad_periodic(__A)[1]
    _gradA = [_gradA_x, _gradA_y]

    _gradA_v = _gradA[0] * _v[0] + _gradA[1] * _v[1]

    _A_fft = torch.fft.fft2(__A)
    _KA_fft = _fK * _A_fft
    _KA = torch.fft.ifft2(_KA_fft)

    _KA_real = _KA.real

    _T_KA = target_torch(_KA_real)

    loss = ((_gradA_v + __A - _T_KA) ** 2).sum()

    return loss

def Loss_torch_param(__A, _v,_m,_s):
    _gradA_x = grad_periodic(__A)[0]
    _gradA_y = grad_periodic(__A)[1]
    _gradA = [_gradA_x, _gradA_y]

    _gradA_v = _gradA[0] * _v[0] + _gradA[1] * _v[1]

    _A_fft = torch.fft.fft2(__A)
    _KA_fft = _fK * _A_fft
    _KA = torch.fft.ifft2(_KA_fft)
    _KA_real = _KA.real
    _T_KA = bell_torch(_KA_real,_m,_s)

    loss = ((_gradA_v + __A - _T_KA) ** 2).sum()

    return loss

def Loss_torch_param_all(__A, _v,_m,_s,_b):
    _gradA_x = grad_periodic(__A)[0]
    _gradA_y = grad_periodic(__A)[1]
    _gradA = [_gradA_x, _gradA_y]

    _gradA_v = _gradA[0] * _v[0] + _gradA[1] * _v[1]

    _A_fft = torch.fft.fft2(__A)
    # _fK = kernel_fft(_b,R,size)
    _fK = mix_kernel_fft_raw(_b)
    _KA_fft = _fK * _A_fft
    _KA = torch.fft.ifft2(_KA_fft)
    _KA_real = _KA.real

    _T_KA = bell_torch(_KA_real,_m,_s)

    loss = ((_gradA_v + __A - _T_KA) ** 2).sum()

    return loss



In [ ]:
# Important parameters (T: size of timestep, resolution: spatial resolution)

T = 10
resolution = 1.0

size = int(144*resolution)
mid = size // 2
R = 36*resolution
Nx, Ny = size, size

x = torch.linspace(0, Nx-1, Nx)
y = torch.linspace(0, Ny-1, Ny)
xv, yv = torch.meshgrid(x, y, indexing='ij')
xv = t(xv)
yv = t(yv)

m = 0.21
s = 0.018
b = np.array([5/6,7/12,1])

# parameters for initial gaussian patterns
Nx, Ny = size, size
x = torch.linspace(0, Nx-1, Nx)
y = torch.linspace(0, Ny-1, Ny)
xv, yv = torch.meshgrid(x, y, indexing='ij')
center_x = Nx//2
center_y = Ny//2

# default kernel function
D = np.linalg.norm(np.array(np.meshgrid(np.arange(-mid,mid), np.arange(-mid,mid))),axis=0)  / R * len(b)
K = (D<len(b)) * b[np.minimum(D.astype(int),len(b)-1)] * bell(D % 1, 0.5, 0.15)
_K = np.fft.ifftshift(K)
fK = np.fft.fft2(_K / np.sum(_K))
_fK = torch.from_numpy(fK).to(device=DEVICE)
# _fK = fK

In [ ]:
# (Optiional) These functions are used to speed up the FFT of the kernel (kernel_fft -> mix_kernel_fft_raw).

def precompute_kernel_fft_basis(num_bins=3, size=size, R=R, sigma=0.15):
    g = torch.arange(size, device=DEVICE) - size // 2
    X, Y = torch.meshgrid(g, g, indexing="ij")
    D = torch.sqrt(X * X + Y * Y) / R * num_bins

    basis_fft, basis_sum = [], []
    for i in range(num_bins):
        mask = (torch.floor(D) == i).float()
        G = mask * torch.exp(-((D % 1 - 0.5) ** 2) / (2 * sigma ** 2))
        basis_fft.append(torch.fft.fft2(torch.fft.ifftshift(G)))
        basis_sum.append(G.sum())

    return torch.stack(basis_fft), torch.tensor(basis_sum, device=DEVICE)

K_FFT_BASIS, G_SUM = precompute_kernel_fft_basis()

def mix_kernel_fft_raw(w, basis_fft=K_FFT_BASIS, basis_sum=G_SUM):
    w_c = w.to(dtype=basis_fft.dtype, device=DEVICE)

    K_fft_un = torch.tensordot(w_c, basis_fft, dims=1)

    total_sum = (w_c * basis_sum).sum()
    K_fft = K_fft_un / total_sum

    return K_fft


In [ ]:
# For generalized fitness function

def velocity_torch_param_all(__A,_m,_s,_b):
    __A_fft  = torch.fft.fft2(__A)
    # __fK = kernel_fft(_b,R,size)
    __fK = mix_kernel_fft_raw(_b)

    __A_conv = torch.real(torch.fft.ifft2(__fK * __A_fft))
    _R = __A - bell_torch(__A_conv,_m,_s)

    __A_x, __A_y = grad_periodic(__A)

    G_xx = torch.sum(__A_x * __A_x)
    G_yy = torch.sum(__A_y * __A_y)
    G_xy = torch.sum(__A_x * __A_y)
    B_x  = torch.sum(_R   * __A_x)
    B_y  = torch.sum(_R   * __A_y)

    detG = G_xx * G_yy - G_xy * G_xy
    eps  = 1e-12
    if torch.abs(detG) < eps:
        raise RuntimeError("Gram matrix is singular.")

    v_x = (B_x * G_yy - B_y * G_xy) / detG
    v_y = (B_y * G_xx - B_x * G_xy) / detG
    return torch.stack((v_x, v_y))

def fitness_torch_param_all_biased(__A,_m,_s,_b,_lambda=1.0,_sigma =0.1):
    # __fK = kernel_fft(_b,R,size)
    __fK = mix_kernel_fft_raw(_b)

    v   = velocity_torch_param_all(__A,_m,_s,_b)
    print(v)
    _R   = __A - bell_torch(torch.real(torch.fft.ifft2(__fK * torch.fft.fft2(__A))),_m,_s)
    __A_x, __A_y = grad_periodic(__A)
    proj = v[0] * __A_x + v[1] * __A_y
    print(proj.sum())
    print(_R.sum())
    eps  = _R - proj
    return torch.linalg.norm(eps) + _lambda * torch.exp(-(v[0] / _sigma) ** 2 / 2)


# Fig 1 : Gradient descent to obtain the glider pattern

In [ ]:
v_x = torch.tensor(3.4*resolution)
v_y = torch.tensor(0.0)
v = [v_x, v_y]

# change this parameter for manipulating initial guassian width
sigma_gaussian = 15*resolution

gaussian = torch.exp(-((xv - center_x)**2 + (yv - center_y)**2) / (2 * sigma_gaussian**2))
A = t(gaussian)

A.requires_grad_(True)

optimizer = optim.AdamW([A], lr=1e-2)
scheduler = StepLR(optimizer, step_size=1000, gamma=0.5)
num_steps = 5000

A_List = []
Loss_List = []
for step in range(num_steps+1):
    optimizer.zero_grad()

    L_A = Loss_torch(A,v)
    L_A.backward()

    optimizer.step()
    scheduler.step()

    if step % 100 == 0:
        # print(f"Step {step}, Loss: {L_A.item()}")
        A_List.append(A.detach().cpu().clone().numpy())
        Loss_List.append(L_A.item())

In [ ]:
fig= plt.figure(figsize=(8,2))
plt.plot(range(0,len(Loss_List)*100,100),Loss_List,marker='.')
plt.yscale('log')
plt.xlabel('step')
plt.ylabel('loss')

In [ ]:
fig= plt.figure(figsize=(10,4))
fig.add_subplot(212)
_Loss_List = Loss_List[:31]
plt.plot(range(0,len(_Loss_List)*100,100),_Loss_List,marker='.')
plt.yscale('log')
plt.xlabel('step')
plt.ylabel('loss')

for i in range(0,7):
  fig.add_subplot(2,7,i+1)
  plt.imshow(A_List[i*5])
  plt.axis('off')
  plt.title('step:'+str(i*500))

In [ ]:
A = A_List[-1]

figure_asset(K, target)
fig = figure_world(A)
IPython.display.HTML(matplotlib.animation.FuncAnimation(fig, update, frames=200, interval=10).to_jshtml())

## optimizing velocity further decreased the loss function

In [ ]:
v_x = torch.tensor(3.4*resolution)
v_y = torch.tensor(0.0)
v = [v_x, v_y]

# change this parameter for manipulating initial guassian width
sigma_gaussian = 15*resolution

gaussian = torch.exp(-((xv - center_x)**2 + (yv - center_y)**2) / (2 * sigma_gaussian**2))
A = t(gaussian)

A.requires_grad_(True)
v_x.requires_grad_(True)

optimizer = optim.AdamW([A,v_x], lr=1e-2)
scheduler = StepLR(optimizer, step_size=1000, gamma=0.5)
num_steps = 8000

A_List = []
Loss_List = []
for step in range(num_steps+1):
    optimizer.zero_grad()

    L_A = Loss_torch(A,v)
    L_A.backward()

    optimizer.step()
    scheduler.step()

    if step % 100 == 0:
        # print(f"Step {step}, Loss: {L_A.item()}")
        A_List.append(A.detach().cpu().clone().numpy())
        Loss_List.append(L_A.item())

In [ ]:
print(v_x)
plt.imshow(A_List[-1])

In [ ]:
fig= plt.figure(figsize=(8,2))
plt.plot(range(0,len(Loss_List)*100,100),Loss_List,marker='.')
plt.yscale('log')
plt.xlabel('step')
plt.ylabel('loss')

# Fig.2 : Gradient descent with different initial conditions

In [ ]:
ResultingPattern = []
for _velocity in [0,2.0,4.0,6.0,8.0,10.0]:
  print(_velocity)
  _ResultingPattern = []
  for _gaussianWidth in [9,12,15,18,21]:
    v_x = torch.tensor(_velocity*resolution)
    v_y = torch.tensor(0.0)
    v = [v_x, v_y]

    sigma_gaussian = _gaussianWidth*resolution
    gaussian = torch.exp(-((xv - center_x)**2 + (yv - center_y)**2) / (2 * sigma_gaussian**2))
    A = t(gaussian)
    A.requires_grad_(True)
    A_List = []

    optimizer = optim.AdamW([A], lr=1e-2)
    scheduler = StepLR(optimizer, step_size=1000, gamma=0.5)
    num_steps = 5000

    for step in range(num_steps+1):
        optimizer.zero_grad()
        L_A = Loss_torch(A,v)
        L_A.backward()

        optimizer.step()
        scheduler.step()

        # if step % 1000 == 0:
        #     print(f"Step {step}, Loss: {L_A.item()}")
        #     A_List.append(A.detach().cpu().clone().numpy())

    _ResultingPattern.append(A.detach().cpu().clone().numpy())
  ResultingPattern.append(_ResultingPattern)

In [ ]:
fig = plt.figure(figsize=(5,6))
for i in range(0,6):
  for j in range(0,5):
    fig.add_subplot(6,5,i*5+j+1)
    plt.imshow(ResultingPattern[i][j],clim=(0,1))
    if i ==5:
      plt.xlabel('width='+str([9,12,15,18,21][j]))
    if j==0:
      plt.ylabel('vel='+str(i*2))
    ax = plt.gca()
    ax.set_xticks([])
    ax.set_yticks([])


In [ ]:
totalTime = 140
totalFrame = T*totalTime

LastImage = []
for _velIdx in range(0,6):
  _LastImage = []
  for _widthIdx in range(0,5):
    A = np.array(ResultingPattern)[_velIdx,_widthIdx]
    for i in range(0,totalFrame):
        update(i)
    _LastImage.append(A)
  LastImage.append(_LastImage)

LastImage = np.array(LastImage)

# Used in Fig.5
LastImage_glider = LastImage[2][2].copy()
LastImage_expand = LastImage[0][4].copy()

In [ ]:
fig = plt.figure(figsize=(5,6))
for i in range(0,6):
  for j in range(0,5):
    fig.add_subplot(6,5,i*5+j+1)
    plt.imshow(LastImage[i][j],clim=(0,1))

    if i ==5:
      plt.xlabel('width='+str([9,12,15,18,21][j]))
    if j==0:
      plt.ylabel('vel='+str(i*2))
    ax = plt.gca()
    ax.set_xticks([])
    ax.set_yticks([])

# Fig.3 : Modifying the glider to speed up

## (a) Only update the initial pattern

In [ ]:
# v_x = torch.tensor(3.4*resolution)
v_x = torch.tensor(6.0*resolution)
v_y = torch.tensor(0.0)
v = [v_x, v_y]

sigma_gaussian = 15*resolution
center_x = Nx//2
center_y = Ny//4
gaussian = torch.exp(-((xv - center_x)**2 + (yv - center_y)**2) / (2 * sigma_gaussian**2))
A = t(gaussian)
A.requires_grad_(True)

A_List = []

optimizer = optim.Adam([
        {"params": [A],   "lr": 1e-2}
    ])
scheduler = StepLR(optimizer, step_size=1000, gamma=0.5)

num_steps = 5000

for step in range(num_steps+1):
    optimizer.zero_grad()

    L_A = Loss_torch(A,v)
    L_A.backward()

    optimizer.step()
    scheduler.step()

    if step % 1000 == 0:
        # print(f"Step {step}, Loss: {L_A.item()}")
        A_List.append(A.detach().cpu().clone().numpy())

In [ ]:
A = A_List[-1]

figure_asset(K, target)
fig = figure_world(A)
IPython.display.HTML(matplotlib.animation.FuncAnimation(fig, update, frames=200, interval=10).to_jshtml())

In [ ]:
totalTime = 40
totalFrame = T*totalTime

A = A_List[-1]
imgList = []
imgList.append(A)
for i in range(0,totalFrame):
    update(i)
    imgList.append(A)

CMList_1 = []
for i in range(0,len(imgList)):
  CMList_1.append(ndimage.center_of_mass(imgList[i]))
CMList_1 = np.array(CMList_1)

fig = plt.figure(figsize=(8,4))
fig.add_subplot(121)
plt.imshow(imgList[0]+imgList[100])
plt.plot(CMList_1[:100,1],CMList_1[:100,0],color='r')
plt.axis('off')
fig.add_subplot(122)
plt.plot(CMList_1)

## (b) Optimization of Bell function

In [ ]:
v_x = torch.tensor(6.0*resolution)
v_y = torch.tensor(0.0)
v = [v_x, v_y]

m_target = torch.tensor(0.21)
s_target = torch.tensor(0.018)

center_x = Nx//2
center_y = Ny//4
sigma_gaussian = 15*resolution
gaussian = torch.exp(-((xv - center_x)**2 + (yv - center_y)**2) / (2 * sigma_gaussian**2))
A = t(gaussian)

A.requires_grad_(True)
m_target.requires_grad_(True)
s_target.requires_grad_(True)

optimizer = optim.Adam([
        {"params": [A],   "lr": 1e-2},
        {"params": [m_target], "lr": 1e-3},
        {"params": [s_target], "lr": 1e-4}
    ])
scheduler = StepLR(optimizer, step_size=1000, gamma=0.5)

num_steps = 5000

A_List = []
for step in range(num_steps+1):
    optimizer.zero_grad()

    L_A = Loss_torch_param(A,v,m_target,s_target)
    L_A.backward()

    optimizer.step()
    scheduler.step()

    if step % 1000 == 0:
        # print(f"Step {step}, Loss: {L_A.item()}")
        A_List.append(A.detach().cpu().clone().numpy())




In [ ]:
_m = m_target.detach().cpu().numpy()
_s = s_target.detach().cpu().numpy()
A = A_List[-1]

figure_asset(K, target)
fig = figure_world(A)
IPython.display.HTML(matplotlib.animation.FuncAnimation(fig, update_param, frames=400, interval=10).to_jshtml())

In [ ]:
totalTime = 40
totalFrame = T*totalTime

_m = m_target.detach().cpu().numpy()
_s = s_target.detach().cpu().numpy()
A = A_List[-1]

imgList = []
imgList.append(A)
for i in range(0,totalFrame):
    update_param(i)
    imgList.append(A)

CMList_2 = []
for i in range(0,len(imgList)):
  CMList_2.append(ndimage.center_of_mass(imgList[i]))
CMList_2 = np.array(CMList_2)

fig = plt.figure(figsize=(8,4))
fig.add_subplot(121)
plt.imshow(imgList[0]+imgList[100])
plt.plot(CMList_2[:100,1],CMList_2[:100,0],color='r')
plt.axis('off')
fig.add_subplot(122)
plt.plot(CMList_2)

## (c) optimization of kernel function

In [ ]:
# v_x = torch.tensor(3.4*resolution)
v_x = torch.tensor(6.0*resolution)
v_y = torch.tensor(0.0)
v = [v_x, v_y]

m_target = torch.tensor(0.21)
s_target = torch.tensor(0.018)

b_kernel = torch.as_tensor(b.copy())

center_x = Nx//2
center_y = Ny//4
sigma_gaussian = 15*resolution
gaussian = torch.exp(-((xv - center_x)**2 + (yv - center_y)**2) / (2 * sigma_gaussian**2))
A = t(gaussian)

A.requires_grad_(True)
m_target.requires_grad_(True)
s_target.requires_grad_(True)
b_kernel.requires_grad_(True)

optimizer = optim.Adam([
        {"params": [A],   "lr": 1e-2},
        {"params": [m_target], "lr": 1e-3},
        {"params": [s_target], "lr": 1e-4},
        {"params": [b_kernel], "lr": 1e-2}
    ])

scheduler = StepLR(optimizer, step_size=1000, gamma=0.5)

num_steps = 5000

A_List = []
for step in range(num_steps+1):
    optimizer.zero_grad()
    L_A = Loss_torch_param_all(A,v,m_target,s_target,b_kernel)
    L_A.backward()

    optimizer.step()
    scheduler.step()

    if step % 1000 == 0:
        # print(f"Step {step}, Loss: {L_A.item()}")
        A_List.append(A.detach().cpu().clone().numpy())


In [ ]:
_b_kernel = b_kernel.detach().cpu().numpy()
D = np.linalg.norm(np.array(np.meshgrid(np.arange(-mid,mid), np.arange(-mid,mid))),axis=0)  / R * len(_b_kernel)
K = (D<len(_b_kernel)) * _b_kernel[np.minimum(D.astype(int),len(_b_kernel)-1)] * bell(D % 1, 0.5, 0.15)
_K=np.fft.ifftshift(K)
fK_new = np.fft.fft2(_K / np.sum(_K))

_m = m_target.detach().cpu().numpy()
_s = s_target.detach().cpu().numpy()

A = A_List[-1]

figure_asset(K, target)
fig = figure_world(A)
IPython.display.HTML(matplotlib.animation.FuncAnimation(fig, update_param_all, frames=400, interval=10).to_jshtml())

In [ ]:
totalTime = 40
T = 10
totalFrame = T*totalTime

_b_kernel = b_kernel.detach().cpu().numpy()
D = np.linalg.norm(np.array(np.meshgrid(np.arange(-mid,mid), np.arange(-mid,mid))),axis=0)  / R * len(_b_kernel)
K = (D<len(_b_kernel)) * _b_kernel[np.minimum(D.astype(int),len(_b_kernel)-1)] * bell(D % 1, 0.5, 0.15)
_K=np.fft.ifftshift(K)
fK_new = np.fft.fft2(_K / np.sum(_K))

_m = m_target.detach().cpu().numpy()
_s = s_target.detach().cpu().numpy()

A = A_List[-1]

imgList = []
imgList.append(A)
for i in range(0,totalFrame):
    update_param_all(i)
    imgList.append(A)

CMList_3 = []
for i in range(0,len(imgList)):
  CMList_3.append(ndimage.center_of_mass(imgList[i]))
CMList_3 = np.array(CMList_3)

fig = plt.figure(figsize=(8,4))
fig.add_subplot(121)
plt.imshow(imgList[0]+imgList[100])
plt.plot(CMList_3[:100,1],CMList_3[:100,0],color='r')
plt.axis('off')
fig.add_subplot(122)
plt.plot(CMList_3)

## Comparison of three trajectories


In [ ]:
fig = plt.figure(figsize=(6,4))
plt.plot(CMList_1[:100,1]-CMList_1[0,1], label='Patern Optimization')
plt.plot(CMList_2[:100:,1]-CMList_2[0,1], label='+ Target function')
plt.plot(CMList_3[:100,1]-CMList_3[0,1], label='+ Kernel function')
plt.legend()
plt.xlabel('Step')
plt.ylabel('Position')


# Fig.4 Search for Glider patterns

In [ ]:
# # This part will take much time
# seed = 3407

# np.random.seed(seed)
# torch.manual_seed(seed)

# ResultingPattern = []
# paramList = []
# for _velocity in [2.0,3.0,4.0,5.0,6.0,8.0,10.0]:
#   v_x = torch.tensor(_velocity*resolution, device=DEVICE)
#   v_y = torch.tensor(0.0, device=DEVICE)
#   v = [v_x, v_y]
#   print(_velocity)
#   for _gaussianWidth in [9,12,15,18]:
#     print(_gaussianWidth)
#     for _outIdx in range(20):
#       print(_outIdx)

#       m_target = torch.tensor(0.21, device=DEVICE)
#       s_target = torch.tensor(0.018, device=DEVICE)
#       b_kernel = torch.rand(3, dtype=torch.float32,device=DEVICE)

#       center_x = Nx//2
#       center_y = Ny//2
#       sigma_gaussian = _gaussianWidth*resolution
#       gaussian = torch.exp(-((xv - center_x)**2 + (yv - center_y)**2) / (2 * sigma_gaussian**2))
#       A = t(gaussian)

#       A.requires_grad_(True)
#       m_target.requires_grad_(True)
#       s_target.requires_grad_(True)
#       b_kernel.requires_grad_(True)

#       A_List = []

#       optimizer = optim.Adam([
#           {"params": [A],   "lr": 1e-2},
#           {"params": [m_target], "lr": 1e-3},
#           {"params": [s_target], "lr": 1e-4},
#           {"params": [b_kernel], "lr": 1e-2}
#       ])
#       scheduler = StepLR(optimizer, step_size=10, gamma=0.5)
#       # scheduler = StepLR(optimizer, step_size=1000, gamma=0.5)
#       num_steps = 5000

#       for step in range(num_steps+1):
#           optimizer.zero_grad()
#           L_A = Loss_torch_param_all(A,v,m_target,s_target,b_kernel)
#           L_A.backward()

#           optimizer.step()

#           if step % 100 == 0:
#             scheduler.step()

#       A_List.append(A.detach().cpu().clone().numpy())

#       ResultingPattern.append(A_List[-1])
#       __b_kernel = b_kernel.detach().cpu().clone().numpy()
#       paramList.append([m_target.detach().cpu().clone().numpy(),s_target.detach().cpu().clone().numpy(),__b_kernel[0],__b_kernel[1],__b_kernel[2],_velocity,_gaussianWidth,L_A.item(),_outIdx])
#       print(__b_kernel)
#       print(L_A.item())
# # np.save(dataDir+'paramList_fig4.npy',np.array(paramList))
# # np.save(dataDir+'ResultingPattern_fig4.npy',np.array(ResultingPattern))

In [ ]:
# totalTime = 100
# totalFrame = T*totalTime

# LastImage = []
# for _index in range(0,len(paramList)):
#   _b_kernel = np.array(paramList[_index][2:5])
#   D = np.linalg.norm(np.array(np.meshgrid(np.arange(-mid,mid), np.arange(-mid,mid))),axis=0)  / R * len(_b_kernel)
#   K = (D<len(_b_kernel)) * _b_kernel[np.minimum(D.astype(int),len(_b_kernel)-1)] * bell(D % 1, 0.5, 0.15)
#   _K=np.fft.ifftshift(K)
#   fK_new = np.fft.fft2(_K / np.sum(_K))

#   _m = paramList[_index][0]
#   _s = paramList[_index][1]

#   A = ResultingPattern[_index]
#   for i in range(0,totalFrame):
#       update_param_all(i)
#   LastImage.append(A)
# LastImage = np.array(LastImage)
# # np.save(dataDir+'LastImage_fig4.npy',LastImage)

In [ ]:
# Load the previously searched patterns (if you have)
paramList = np.load(dataDir+'paramList_fig4.npy')
ResultingPattern = np.load(dataDir+'ResultingPattern_fig4.npy')
LastImage = np.load(dataDir+'LastImage_fig4.npy')

In [ ]:
_nonZero = LastImage.sum(axis=(1,2))>0.1

fig = plt.figure(figsize=(10,4))

_velocity = 3.0
_gaussianWidth = 15
_param = paramList[(paramList[:,6]==_gaussianWidth) & (paramList[:,5]==_velocity)&_nonZero]
_pattern = ResultingPattern[(paramList[:,6]==_gaussianWidth) & (paramList[:,5]==_velocity)&_nonZero]
_pattern_last = LastImage[(paramList[:,6]==_gaussianWidth) & (paramList[:,5]==_velocity)&_nonZero]


_idx = np.argsort(_param[:,7])

for i in range(0,5):
  fig.add_subplot(2,5,i+1)
  if i==0:
    plt.ylabel("Obtained Pattern")
  plt.imshow(_pattern[_idx[i]],clim=(0,1))
  ax = plt.gca()
  ax.set_xticks([])
  ax.set_yticks([])
fig.suptitle("velocity = {:.1f}, gaussianWidth = {}".format(_velocity,_gaussianWidth))

for i in range(0,5):
  fig.add_subplot(2,5,i+6)
  if i==0:
    plt.ylabel("After 100 steps")
  plt.imshow(_pattern_last[_idx[i]],clim=(0,1))
  # plt.axis('off')
  plt.xlabel("Loss:{:.1e}".format(_param[_idx[i],7]))
  ax = plt.gca()
  ax.set_xticks([])
  ax.set_yticks([])
fig.suptitle("velocity = {:.1f}, gaussianWidth = {}".format(_velocity,_gaussianWidth))


In [ ]:
_nonZero = LastImage.sum(axis=(1,2))>0.1

fig = plt.figure(figsize=(10,4))

_velocity = 4.0
_gaussianWidth = 15
_param = paramList[(paramList[:,6]==_gaussianWidth) & (paramList[:,5]==_velocity)&_nonZero]
_pattern = ResultingPattern[(paramList[:,6]==_gaussianWidth) & (paramList[:,5]==_velocity)&_nonZero]
_pattern_last = LastImage[(paramList[:,6]==_gaussianWidth) & (paramList[:,5]==_velocity)&_nonZero]


_idx = np.argsort(_param[:,7])

for i in range(0,5):
  fig.add_subplot(2,5,i+1)
  if i==0:
    plt.ylabel("Obtained Pattern")
  plt.imshow(_pattern[_idx[i]],clim=(0,1))
  ax = plt.gca()
  ax.set_xticks([])
  ax.set_yticks([])
fig.suptitle("velocity = {:.1f}, gaussianWidth = {}".format(_velocity,_gaussianWidth))

for i in range(0,5):
  fig.add_subplot(2,5,i+6)
  if i==0:
    plt.ylabel("After 100 steps")
  plt.imshow(_pattern_last[_idx[i]],clim=(0,1))
  # plt.axis('off')
  plt.xlabel("Loss:{:.1e}".format(_param[_idx[i],7]))
  ax = plt.gca()
  ax.set_xticks([])
  ax.set_yticks([])
fig.suptitle("velocity = {:.1f}, gaussianWidth = {}".format(_velocity,_gaussianWidth))


In [ ]:
# # Save the images of generated patterns
# fig = plt.figure(figsize=(10,8))

# for _velocity in [2.0,3.0,4.0,5.0,6.0,8.0,10.0]:
#   for _gaussianWidth in [9,12,15,18]:
#     _param = paramList[(paramList[:,6]==_gaussianWidth) & (paramList[:,5]==_velocity)]
#     _pattern = np.array(ResultingPattern)[(paramList[:,6]==_gaussianWidth) & (paramList[:,5]==_velocity)]
#     _idx = np.argsort(_param[:,7])

#     for i in range(0,20):
#       fig.add_subplot(4,5,i+1)
#       plt.imshow(_pattern[_idx[i]],clim=(0,1))
#       plt.axis('off')
#       plt.title("Loss:{:.1e}".format(_param[_idx[i],7]))
#     fig.suptitle("velocity = {:.1f}, gaussianWidth = {}".format(_velocity,_gaussianWidth))
#     fig.savefig(dataDir+"Fig_2/Patterns_velocity_{:.1f}_gaussianWidth_{}.png".format(_velocity,_gaussianWidth))
#     fig.clf()

# _velIdx = -1
# for _velocity in [2.0,3.0,4.0,5.0,6.0,8.0,10.0]:
#   _velIdx = _velIdx + 1
#   _widthIdx = -1
#   for _gaussianWidth in [9,12,15,18]:
#     _widthIdx = _widthIdx + 1
#     _param = paramList[(paramList[:,6]==_gaussianWidth) & (paramList[:,5]==_velocity)]
#     _pattern = LastImage[(paramList[:,6]==_gaussianWidth) & (paramList[:,5]==_velocity)]
#     _idx = np.argsort(_param[:,7])

#     for i in range(0,20):
#       fig.add_subplot(4,5,i+1)
#       plt.imshow(_pattern[_idx[i]],clim=(0,1))
#       plt.axis('off')
#       plt.title("{:.1e}, {}".format(_param[_idx[i],7],_idx[i]+20*(_velIdx*4+_widthIdx)))
#     fig.suptitle("velocity = {:.1f}, gaussianWidth = {}".format(_velocity,_gaussianWidth))
#     fig.savefig(dataDir+"Fig_2/ConvergedPatterns_velocity_{:.1f}_gaussianWidth_{}_250508_2.png".format(_velocity,_gaussianWidth))
#     fig.clf()

## Example videos of several obtained patterns

### Obtained from v=3.0 & gaussianWidth = 15

In [ ]:
_index = 135

_b_kernel = np.array(paramList[_index][2:5])
D = np.linalg.norm(np.array(np.meshgrid(np.arange(-mid,mid), np.arange(-mid,mid))),axis=0)  / R * len(_b_kernel)
K = (D<len(_b_kernel)) * _b_kernel[np.minimum(D.astype(int),len(_b_kernel)-1)] * bell(D % 1, 0.5, 0.15)
_K=np.fft.ifftshift(K)
fK_new = np.fft.fft2(_K / np.sum(_K))

_m = paramList[_index][0]
_s = paramList[_index][1]

A = ResultingPattern[_index]


figure_asset(K, target)
fig = figure_world(A)
IPython.display.HTML(matplotlib.animation.FuncAnimation(fig, update_param_all, frames=400, interval=10).to_jshtml())



In [ ]:
_index = 124

_b_kernel = np.array(paramList[_index][2:5])
D = np.linalg.norm(np.array(np.meshgrid(np.arange(-mid,mid), np.arange(-mid,mid))),axis=0)  / R * len(_b_kernel)
K = (D<len(_b_kernel)) * _b_kernel[np.minimum(D.astype(int),len(_b_kernel)-1)] * bell(D % 1, 0.5, 0.15)
_K=np.fft.ifftshift(K)
fK_new = np.fft.fft2(_K / np.sum(_K))

_m = paramList[_index][0]
_s = paramList[_index][1]

A = ResultingPattern[_index]


figure_asset(K, target)
fig = figure_world(A)
IPython.display.HTML(matplotlib.animation.FuncAnimation(fig, update_param_all, frames=400, interval=10).to_jshtml())



In [ ]:
_index = 132

_b_kernel = np.array(paramList[_index][2:5])
D = np.linalg.norm(np.array(np.meshgrid(np.arange(-mid,mid), np.arange(-mid,mid))),axis=0)  / R * len(_b_kernel)
K = (D<len(_b_kernel)) * _b_kernel[np.minimum(D.astype(int),len(_b_kernel)-1)] * bell(D % 1, 0.5, 0.15)
_K=np.fft.ifftshift(K)
fK_new = np.fft.fft2(_K / np.sum(_K))

_m = paramList[_index][0]
_s = paramList[_index][1]

A = ResultingPattern[_index]


figure_asset(K, target)
fig = figure_world(A)
IPython.display.HTML(matplotlib.animation.FuncAnimation(fig, update_param_all, frames=400, interval=10).to_jshtml())



In [ ]:
_index = 128

_b_kernel = np.array(paramList[_index][2:5])
D = np.linalg.norm(np.array(np.meshgrid(np.arange(-mid,mid), np.arange(-mid,mid))),axis=0)  / R * len(_b_kernel)
K = (D<len(_b_kernel)) * _b_kernel[np.minimum(D.astype(int),len(_b_kernel)-1)] * bell(D % 1, 0.5, 0.15)
_K=np.fft.ifftshift(K)
fK_new = np.fft.fft2(_K / np.sum(_K))

_m = paramList[_index][0]
_s = paramList[_index][1]

A = ResultingPattern[_index]


figure_asset(K, target)
fig = figure_world(A)
IPython.display.HTML(matplotlib.animation.FuncAnimation(fig, update_param_all, frames=400, interval=10).to_jshtml())



In [ ]:
_index = 133

_b_kernel = np.array(paramList[_index][2:5])
D = np.linalg.norm(np.array(np.meshgrid(np.arange(-mid,mid), np.arange(-mid,mid))),axis=0)  / R * len(_b_kernel)
K = (D<len(_b_kernel)) * _b_kernel[np.minimum(D.astype(int),len(_b_kernel)-1)] * bell(D % 1, 0.5, 0.15)
_K=np.fft.ifftshift(K)
fK_new = np.fft.fft2(_K / np.sum(_K))

_m = paramList[_index][0]
_s = paramList[_index][1]

A = ResultingPattern[_index]


figure_asset(K, target)
fig = figure_world(A)
IPython.display.HTML(matplotlib.animation.FuncAnimation(fig, update_param_all, frames=400, interval=10).to_jshtml())



### Obtained from v=4.0 & gaussianWidth = 15

In [ ]:
_index = 203

_b_kernel = np.array(paramList[_index][2:5])
D = np.linalg.norm(np.array(np.meshgrid(np.arange(-mid,mid), np.arange(-mid,mid))),axis=0)  / R * len(_b_kernel)
K = (D<len(_b_kernel)) * _b_kernel[np.minimum(D.astype(int),len(_b_kernel)-1)] * bell(D % 1, 0.5, 0.15)
_K=np.fft.ifftshift(K)
fK_new = np.fft.fft2(_K / np.sum(_K))

_m = paramList[_index][0]
_s = paramList[_index][1]

A = ResultingPattern[_index]


figure_asset(K, target)
fig = figure_world(A)
IPython.display.HTML(matplotlib.animation.FuncAnimation(fig, update_param_all, frames=400, interval=10).to_jshtml())



In [ ]:
_index = 202

_b_kernel = np.array(paramList[_index][2:5])
D = np.linalg.norm(np.array(np.meshgrid(np.arange(-mid,mid), np.arange(-mid,mid))),axis=0)  / R * len(_b_kernel)
K = (D<len(_b_kernel)) * _b_kernel[np.minimum(D.astype(int),len(_b_kernel)-1)] * bell(D % 1, 0.5, 0.15)
_K=np.fft.ifftshift(K)
fK_new = np.fft.fft2(_K / np.sum(_K))

_m = paramList[_index][0]
_s = paramList[_index][1]

A = ResultingPattern[_index]


figure_asset(K, target)
fig = figure_world(A)
IPython.display.HTML(matplotlib.animation.FuncAnimation(fig, update_param_all, frames=400, interval=10).to_jshtml())



In [ ]:
_index = 209

_b_kernel = np.array(paramList[_index][2:5])
D = np.linalg.norm(np.array(np.meshgrid(np.arange(-mid,mid), np.arange(-mid,mid))),axis=0)  / R * len(_b_kernel)
K = (D<len(_b_kernel)) * _b_kernel[np.minimum(D.astype(int),len(_b_kernel)-1)] * bell(D % 1, 0.5, 0.15)
_K=np.fft.ifftshift(K)
fK_new = np.fft.fft2(_K / np.sum(_K))

_m = paramList[_index][0]
_s = paramList[_index][1]

A = ResultingPattern[_index]


figure_asset(K, target)
fig = figure_world(A)
IPython.display.HTML(matplotlib.animation.FuncAnimation(fig, update_param_all, frames=400, interval=10).to_jshtml())



In [ ]:
_index = 212

_b_kernel = np.array(paramList[_index][2:5])
D = np.linalg.norm(np.array(np.meshgrid(np.arange(-mid,mid), np.arange(-mid,mid))),axis=0)  / R * len(_b_kernel)
K = (D<len(_b_kernel)) * _b_kernel[np.minimum(D.astype(int),len(_b_kernel)-1)] * bell(D % 1, 0.5, 0.15)
_K=np.fft.ifftshift(K)
fK_new = np.fft.fft2(_K / np.sum(_K))

_m = paramList[_index][0]
_s = paramList[_index][1]

A = ResultingPattern[_index]


figure_asset(K, target)
fig = figure_world(A)
IPython.display.HTML(matplotlib.animation.FuncAnimation(fig, update_param_all, frames=400, interval=10).to_jshtml())



In [ ]:
_index = 219

_b_kernel = np.array(paramList[_index][2:5])
D = np.linalg.norm(np.array(np.meshgrid(np.arange(-mid,mid), np.arange(-mid,mid))),axis=0)  / R * len(_b_kernel)
K = (D<len(_b_kernel)) * _b_kernel[np.minimum(D.astype(int),len(_b_kernel)-1)] * bell(D % 1, 0.5, 0.15)
_K=np.fft.ifftshift(K)
fK_new = np.fft.fft2(_K / np.sum(_K))

_m = paramList[_index][0]
_s = paramList[_index][1]

A = ResultingPattern[_index]


figure_asset(K, target)
fig = figure_world(A)
IPython.display.HTML(matplotlib.animation.FuncAnimation(fig, update_param_all, frames=400, interval=10).to_jshtml())



### Other patterns

In [ ]:
_index = 509

_b_kernel = np.array(paramList[_index][2:5])
D = np.linalg.norm(np.array(np.meshgrid(np.arange(-mid,mid), np.arange(-mid,mid))),axis=0)  / R * len(_b_kernel)
K = (D<len(_b_kernel)) * _b_kernel[np.minimum(D.astype(int),len(_b_kernel)-1)] * bell(D % 1, 0.5, 0.15)
_K=np.fft.ifftshift(K)
fK_new = np.fft.fft2(_K / np.sum(_K))

_m = paramList[_index][0]
_s = paramList[_index][1]

A = ResultingPattern[_index]


figure_asset(K, target)
fig = figure_world(A)
IPython.display.HTML(matplotlib.animation.FuncAnimation(fig, update_param_all, frames=400, interval=10).to_jshtml())



In [ ]:
_index = 181

_b_kernel = np.array(paramList[_index][2:5])
D = np.linalg.norm(np.array(np.meshgrid(np.arange(-mid,mid), np.arange(-mid,mid))),axis=0)  / R * len(_b_kernel)
K = (D<len(_b_kernel)) * _b_kernel[np.minimum(D.astype(int),len(_b_kernel)-1)] * bell(D % 1, 0.5, 0.15)
_K=np.fft.ifftshift(K)
fK_new = np.fft.fft2(_K / np.sum(_K))

_m = paramList[_index][0]
_s = paramList[_index][1]

A = ResultingPattern[_index]


figure_asset(K, target)
fig = figure_world(A)
IPython.display.HTML(matplotlib.animation.FuncAnimation(fig, update_param_all, frames=200, interval=10).to_jshtml())



In [ ]:
_index = 292

_b_kernel = np.array(paramList[_index][2:5])
D = np.linalg.norm(np.array(np.meshgrid(np.arange(-mid,mid), np.arange(-mid,mid))),axis=0)  / R * len(_b_kernel)
K = (D<len(_b_kernel)) * _b_kernel[np.minimum(D.astype(int),len(_b_kernel)-1)] * bell(D % 1, 0.5, 0.15)
_K=np.fft.ifftshift(K)
fK_new = np.fft.fft2(_K / np.sum(_K))

_m = paramList[_index][0]
_s = paramList[_index][1]

A = ResultingPattern[_index]


figure_asset(K, target)
fig = figure_world(A)
IPython.display.HTML(matplotlib.animation.FuncAnimation(fig, update_param_all, frames=500, interval=10).to_jshtml())



# Fig.5 : Estimation of the velocity of the glider only from the single image

In [ ]:
m_target = torch.tensor(0.21)
s_target = torch.tensor(0.018)
b_kernel = torch.as_tensor(b.copy())

fig = plt.figure(figsize=(10,3))

fig.add_subplot(131)
center_x = Nx//2
center_y = Ny//2
sigma_gaussian = 15*resolution
gaussian = torch.exp(-((xv - center_x)**2 + (yv - center_y)**2) / (2 * sigma_gaussian**2))
plt.imshow(gaussian.detach().cpu().numpy())
ax = plt.gca()
ax.set_xticks([])
ax.set_yticks([])

_v = velocity_torch_param_all(t(gaussian),m_target,s_target,b_kernel)[0]
_fitness = fitness_torch_param_all_biased(t(gaussian),m_target,s_target,b_kernel,0,0)
plt.title('Gaussian')
plt.xlabel('v = {:.2f}, fitness ={:.2f}'.format(abs(_v),_fitness))

fig.add_subplot(132)
plt.imshow(LastImage_glider)
ax = plt.gca()
ax.set_xticks([])
ax.set_yticks([])
_v = velocity_torch_param_all(torch.as_tensor(LastImage_glider,device=DEVICE),m_target,s_target,b_kernel)[0]
_fitness = fitness_torch_param_all_biased(torch.as_tensor(LastImage_glider,device=DEVICE),m_target,s_target,b_kernel,0,0)

plt.title('Glider')
plt.xlabel('v = {:.2f}, fitness ={:.2f}'.format(abs(_v),_fitness))


fig.add_subplot(133)
plt.imshow(LastImage_expand)
ax = plt.gca()
ax.set_xticks([])
ax.set_yticks([])
_v = velocity_torch_param_all(torch.as_tensor(LastImage_expand,device=DEVICE),m_target,s_target,b_kernel)[0]
_fitness = fitness_torch_param_all_biased(torch.as_tensor(LastImage_expand,device=DEVICE),m_target,s_target,b_kernel,0,0)
plt.title('Stationary Pattern')
plt.xlabel('v = {:.2f}, fitness ={:.2f}'.format(abs(_v),_fitness))


# Fig.6 : Gradient descent on the "velocity-free" fitness function

In [ ]:
seed = 29

np.random.seed(seed)
torch.manual_seed(seed)

m_target = torch.tensor(0.21)
s_target = torch.tensor(0.018)
b_kernel = torch.rand(3, dtype=torch.float32,device=DEVICE)

center_x = Nx//2
center_y = Ny//2
sigma_gaussian = 15*resolution
gaussian = torch.exp(-((xv - center_x)**2 + (yv - center_y)**2) / (2 * sigma_gaussian**2))
A = t(gaussian)

A.requires_grad_(True)
m_target.requires_grad_(True)
s_target.requires_grad_(True)
b_kernel.requires_grad_(True)

num_steps = 5000

optimizer = optim.Adam([
        {"params": [A],   "lr": 1e-2},
        {"params": [m_target], "lr": 1e-3},
        {"params": [s_target], "lr": 1e-4},
        {"params": [b_kernel], "lr": 1e-2}
    ])
scheduler = StepLR(optimizer, step_size=1000, gamma=0.5)


A_List = []
paramList = []
for step in range(num_steps+1):
    optimizer.zero_grad()
    L_A = fitness_torch_param_all_biased(A,m_target,s_target,b_kernel,_lambda = 1.0,_sigma=0.1)
    L_A.backward()

    optimizer.step()
    scheduler.step()

    if step % 1000 == 0:
      print(f"Step {step}, Loss: {L_A.item()}")

A_List.append(A.detach().cpu().clone().numpy())
paramList.append([m_target.detach().cpu().clone().numpy(),s_target.detach().cpu().clone().numpy(),b_kernel.detach().cpu().clone().numpy()])



In [ ]:
plt.imshow(A_List[-1])

## Generated Examples

In [ ]:
# Generated example (seed = 29)
_index = -1

_b_kernel = paramList[_index][2]
D = np.linalg.norm(np.array(np.meshgrid(np.arange(-mid,mid), np.arange(-mid,mid))),axis=0)  / R * len(_b_kernel)
K = (D<len(_b_kernel)) * _b_kernel[np.minimum(D.astype(int),len(_b_kernel)-1)] * bell(D % 1, 0.5, 0.15)
_K=np.fft.ifftshift(K)
fK_new = np.fft.fft2(_K / np.sum(_K))

_m = paramList[_index][0]
_s = paramList[_index][1]

A = A_List[_index]


figure_asset(K, target)
fig = figure_world(A)
IPython.display.HTML(matplotlib.animation.FuncAnimation(fig, update_param_all, frames=600, interval=10).to_jshtml())



In [ ]:
# Generated example (seed = 1)
_index = -1

_b_kernel = paramList[_index][2]
D = np.linalg.norm(np.array(np.meshgrid(np.arange(-mid,mid), np.arange(-mid,mid))),axis=0)  / R * len(_b_kernel)
K = (D<len(_b_kernel)) * _b_kernel[np.minimum(D.astype(int),len(_b_kernel)-1)] * bell(D % 1, 0.5, 0.15)
_K=np.fft.ifftshift(K)
fK_new = np.fft.fft2(_K / np.sum(_K))

_m = paramList[_index][0]
_s = paramList[_index][1]

A = A_List[_index]


figure_asset(K, target)
fig = figure_world(A)
IPython.display.HTML(matplotlib.animation.FuncAnimation(fig, update_param_all, frames=400, interval=10).to_jshtml())



In [ ]:
# Generated example (seed = 3407)

_index = -1

_b_kernel = paramList[_index][2]
D = np.linalg.norm(np.array(np.meshgrid(np.arange(-mid,mid), np.arange(-mid,mid))),axis=0)  / R * len(_b_kernel)
K = (D<len(_b_kernel)) * _b_kernel[np.minimum(D.astype(int),len(_b_kernel)-1)] * bell(D % 1, 0.5, 0.15)
_K=np.fft.ifftshift(K)
fK_new = np.fft.fft2(_K / np.sum(_K))

_m = paramList[_index][0]
_s = paramList[_index][1]

A = A_List[_index]


figure_asset(K, target)
fig = figure_world(A)
IPython.display.HTML(matplotlib.animation.FuncAnimation(fig, update_param_all, frames=400, interval=10).to_jshtml())



In [ ]:
# Generated example (seed = 2)
_index = -1

_b_kernel = paramList[_index][2]
D = np.linalg.norm(np.array(np.meshgrid(np.arange(-mid,mid), np.arange(-mid,mid))),axis=0)  / R * len(_b_kernel)
K = (D<len(_b_kernel)) * _b_kernel[np.minimum(D.astype(int),len(_b_kernel)-1)] * bell(D % 1, 0.5, 0.15)
_K=np.fft.ifftshift(K)
fK_new = np.fft.fft2(_K / np.sum(_K))

_m = paramList[_index][0]
_s = paramList[_index][1]

A = A_List[_index]


figure_asset(K, target)
fig = figure_world(A)
IPython.display.HTML(matplotlib.animation.FuncAnimation(fig, update_param_all, frames=200, interval=10).to_jshtml())

